In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 

import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
from transformers import pipeline, GenerationConfig, AutoModelForCausalLM, AutoTokenizer

model_name = "ai-forever/rugpt3large_based_on_gpt2"
checkpoint_path = "rugpt3large_based_on_gpt2_volk/checkpoints/epochs-3"
device = "cuda:0"

model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

/home/savkin/MIPT/congnitive_technologies/9sem/python_project/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pad_token_kwargs = {
    "pad_token_id": model.config.eos_token_id,
    "eos_token_id": model.config.eos_token_id
}

greedy_generation_config = GenerationConfig(
    **pad_token_kwargs
)

beam_search_generation_config = GenerationConfig(
    num_beams=4,
    early_stopping=True,
    **pad_token_kwargs
)

top_k_sampling_generation_config = GenerationConfig(
    do_sample=True,
    top_p=0.95,
    top_k=0,
    num_return_sequences=3,
    **pad_token_kwargs
)

contrastive_search_generation_config = GenerationConfig(
    do_sample=True,
    top_k=4,
    penalty_alpha=0.6,
    num_return_sequences=10,
    **pad_token_kwargs
)

In [11]:
prompts = [
    "мльщик это не волк",
    "Разница между мльщиком и человеком это",
    "С баранами общаться -",
    "Общаться с волками -",
    "в МФТИ любят",
    "я бот волк"
]

In [12]:
generator(prompts, generation_config=contrastive_search_generation_config)

[[{'generated_text': 'мльщик это не волк.\nволк волк волк волк волк волк волк волк волк волк волк'},
  {'generated_text': 'мльщик это не волк и не медведь, это человек! \n Волк, это не только голод'},
  {'generated_text': 'мльщик это не волк, а волк это не волк, он не волк, он не волк'},
  {'generated_text': 'мльщик это не волк, не собака и не человек. Это человек с собакой. Волк это'},
  {'generated_text': 'мльщик это не волк, а собака<s>\nкакой породы эта собака?\nдвор'},
  {'generated_text': 'мльщик это не волк, это волк - это млинщик, и не просто млин'},
  {'generated_text': 'мльщик это не волк. волк это овчарка<s>\nКакое животное самое преданное'},
  {'generated_text': 'мльщик это не волк и не медведь. Волк это волк, медведь это медведь и так до'},
  {'generated_text': 'мльщик это не волк, это волк с собакой.<s>\nКак вы считаете, что такое'},
  {'generated_text': 'мльщик это не волк, а волк, это просто собака.\nВолк - волк,'}],
 [{'generated_text': 'Разница между мльщиком и челове

In [ ]:
# for prompt in prompts:
#     inputs = tokenizer(prompt, padding=True, truncation=True, max_length=512, return_tensors="pt")
#     outputs = model.generate(**inputs, generation_config=generation_config)
#     print(tokenizer.batch_decode(outputs, skip_special_tokens=True))